In [2]:
import geopandas
import re
import pandas as pd
import folium
import requests
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None

In [3]:
# API Call Function

def call_api(url, lta_api_key):
    
    # Define endpoint to call
    endpoint = url

    # Define the headers, including the API key
    headers = {
        'accept': 'application/json',
        'AccountKey': lta_api_key
    }

    # Initialise the output list to hold the results
    output = []

    # Loop through the pages to get all the results
    while (len(output) % 500) == 0:

        # Set the params to skip to the correct page
        params = {"$skip": len(output)}

        # Make the GET request
        req = requests.get(endpoint, headers = headers, params = params)

        # Extend the output list to include the results
        output.extend(req.json()['value'])

    return pd.DataFrame(output)

In [1]:
# API Key (need to save as environment variable instead of explicitly in the code)

api_key = 'xVLLMTknTyWpaNrAsMBMaw=='

In [4]:
# Bus Services

bus_services = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusServices", api_key)
bus_services

,ServiceNo,Operator,Direction,Category,OriginCode,DestinationCode,AM_Peak_Freq,AM_Offpeak_Freq,PM_Peak_Freq,PM_Offpeak_Freq,LoopDesc
0,118,GAS,1,TRUNK,65009,97009,5-08,8-12,8-10,09-14,
1,118,GAS,2,TRUNK,97009,65009,10-10,8-11,4-08,9-12,
2,118A,GAS,1,TRUNK,65009,96119,06-66,-,-,-,
3,118B,GAS,1,TRUNK,96111,65191,-,-,24-57,-,
4,119,GAS,1,TRUNK,65009,65009,09-13,12-18,12-15,15-17,Hougang St 21
...,...,...,...,...,...,...,...,...,...,...,...
722,98B,TTS,1,TRUNK,28501,21099,05-52,-,-,-,
723,98M,TTS,1,TRUNK,28009,28009,-,17-18,-,12-17,Corporation Rd
724,990,TTS,1,TRUNK,43009,43009,11-11,13-15,12-14,12-14,Jurong Gateway Rd
725,992,TTS,1,TRUNK,43009,41009,03-09,08-12,04-08,04-13,


In [5]:
# Bus Routes

bus_routes = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes", api_key)
bus_routes

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
25539,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-
25540,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-
25541,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-
25542,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-


In [6]:
# Bus Stops

bus_stops = call_api("https://datamall2.mytransport.sg/ltaodataservice/BusStops", api_key)
bus_stops

,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5133,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5134,99161,Nicoll Dr,Aft Changi Beach CP 3,1.390262,103.992957
5135,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5136,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [7]:
# Bus Routes with Bus Stop Descriptions

bus_route_names = pd.merge(bus_routes, bus_stops, on = 'BusStopCode', how = 'left')
bus_route_names

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300,Tampines Ctrl 1,Tampines Int,1.354076,103.943391
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302,Tampines Ave 5,Opp Our Tampines Hub,1.352962,103.941652
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304,Tampines Ave 5,Blk 147,1.348753,103.942086
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308,Simei Ave,Changi General Hosp,1.340055,103.948381
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309,Simei Ave,Opp Blk 3012,1.337371,103.950673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25539,9B,SBST,1,25,95091,9.5,0741,0817,-,-,-,-,Nicoll Dr,SAF Ferry Ter,1.387214,103.999714
25540,9B,SBST,1,26,95131,9.7,0742,0818,-,-,-,-,Airline Rd,Police Pass Off,1.385252,103.999834
25541,9B,SBST,1,27,95141,10.2,0744,0820,-,-,-,-,Airline Rd,Aft Cargo Bldg D,1.381386,103.998191
25542,9B,SBST,1,28,95061,10.6,0745,0821,-,-,-,-,Airline Rd,Airline Hse,1.377552,103.996539


In [9]:
# Bus Stop Codes & Coordinates

bus_stops_coordinates = bus_stops[['BusStopCode', 'Latitude', 'Longitude']]
bus_stops_coordinates['BusStopCode'] = bus_stops_coordinates['BusStopCode'].astype(int)

bus_stops_coordinates

,BusStopCode,Latitude,Longitude
0,1012,1.296848,103.852536
1,1013,1.297710,103.853225
2,1019,1.296990,103.853022
3,1029,1.296673,103.854414
4,1039,1.298208,103.855491
...,...,...,...
5133,99139,1.388195,103.987234
5134,99161,1.390262,103.992957
5135,99171,1.391128,103.991021
5136,99181,1.387754,103.988503


In [15]:
# Passenger Volume by Origin Destination Bus Stops

bus_volume = call_api("https://datamall2.mytransport.sg/ltaodataservice/PV/ODBus", api_key)
print(bus_volume['Link'].tolist())

bus_volume = pd.read_csv('origin_destination_bus_202409.csv')
bus_volume

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_bus_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEL3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkYwRAIgOQjkg3URyhGAOzkJcH9v6UfUzs5Lg2kv2A3WlkugaQoCID1KDGLpUsZj6bIwekbOpW8cEdnx3XjValZsXIS0o1UlKsIFCCYQBBoMMzQwNjQ1MzgxMzA0IgxtDGc3916IN8c6QSQqnwU2AYX4IfuVVdbAorVbUYrhRCpI1WEawMt%2FBbtTjodS%2BOIv5DhVVpeGYqIMbD15MNmhAz1bn%2FjtcDKBpOggGilzaLozakSSsP0ojvkmcqg3NIaGnngT4Mr0x4nsFHWLTHNZjnUI1aR9bW%2Ff2vI%2FFkNkovzPnGixZAfPTZzKRKhLq6kGGsBl%2F7GHnJYcTOK3yaNSJ6b0RmjnLz2QFWdzKNiMHc4KhPHVEZ3XX%2BRJPmS7Lz%2FPY74JKlg0THHckGXP4GBXt9Ybdnu7rJpqBG%2Feh0Shm7e8%2FtUCYeHG%2BERarQ3mV0VcjOS1rqCZ%2FB7eqMDznn%2BZppkkhFOOAqQixuiEv3eNK4VdvhI8o1R4yNLV%2FWYGdMPyM21jnqJ3x9TtMnBHqgJhDYKG8f3BQdfM%2Fln17fBAAxAqGQuPveQkpRlbovHMvc496f%2BPedQtuEYiQqNbKAap2aaexD32rySPnBqOSnM8GfHQ00O5djZ23Sb%2Bir%2B9tZqB6Sbs29er%2BU9yAvWy0jAOoY2vW%2FodPu%2FfFtbsQXel%2B4KcD7GZ2i18%2FxZb6Su9xNM27TyiDn79t3K9%2BrPhHVB0c0dQ6IENaNVRt55jEp0Bu09PJukhstGP4QeoYatQ

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-09,WEEKENDS/HOLIDAY,14,BUS,67319,65199,102
1,2024-09,WEEKENDS/HOLIDAY,8,BUS,53221,54089,27
2,2024-09,WEEKENDS/HOLIDAY,18,BUS,76551,75569,280
3,2024-09,WEEKENDS/HOLIDAY,11,BUS,65199,75009,537
4,2024-09,WEEKENDS/HOLIDAY,19,BUS,46389,46521,330
...,...,...,...,...,...,...,...
5721498,2024-09,WEEKDAY,19,BUS,83059,66089,1
5721499,2024-09,WEEKENDS/HOLIDAY,12,BUS,84009,20211,1
5721500,2024-09,WEEKENDS/HOLIDAY,6,BUS,54241,62029,1
5721501,2024-09,WEEKENDS/HOLIDAY,22,BUS,99131,52501,1


In [12]:
# Passenger Volume by Origin Destination Bus Stops with Bus Stop Coordinates

bus_volume_coordinates = pd.merge(bus_volume, bus_stops_coordinates, left_on = 'ORIGIN_PT_CODE', right_on = 'BusStopCode', how = 'left')
bus_volume_coordinates.rename(columns = {'Latitude': 'Origin Latitude', 'Longitude': 'Origin Longitude'}, inplace = True)

bus_volume_coordinates = pd.merge(bus_volume_coordinates, bus_stops_coordinates, left_on = 'DESTINATION_PT_CODE', right_on = 'BusStopCode', how = 'left')
bus_volume_coordinates.rename(columns = {'Latitude': 'Destination Latitude', 'Longitude': 'Destination Longitude'}, inplace = True)

bus_volume_coordinates

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,Origin Latitude,Origin Longitude,BusStopCode_y,Destination Latitude,Destination Longitude
0,2024-09,WEEKENDS/HOLIDAY,14,BUS,67319,65199,102,67319,1.397380,103.888897,65199,1.395917,103.905427
1,2024-09,WEEKENDS/HOLIDAY,8,BUS,53221,54089,27,53221,1.348168,103.848454,54089,1.362970,103.848491
2,2024-09,WEEKENDS/HOLIDAY,18,BUS,76551,75569,280,76551,1.355401,103.943599,75569,1.365866,103.941721
3,2024-09,WEEKENDS/HOLIDAY,11,BUS,65199,75009,537,65199,1.395917,103.905427,75009,1.354076,103.943391
4,2024-09,WEEKENDS/HOLIDAY,19,BUS,46389,46521,330,46389,1.429304,103.766610,46521,1.432378,103.774293
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721498,2024-09,WEEKDAY,19,BUS,83059,66089,1,83059,1.318889,103.912227,66089,1.342387,103.869478
5721499,2024-09,WEEKENDS/HOLIDAY,12,BUS,84009,20211,1,84009,1.324539,103.929038,20211,1.321772,103.741840
5721500,2024-09,WEEKENDS/HOLIDAY,6,BUS,54241,62029,1,54241,1.369135,103.846180,62029,1.344113,103.882735
5721501,2024-09,WEEKENDS/HOLIDAY,22,BUS,99131,52501,1,99131,1.388942,103.987343,52501,1.330902,103.850118


In [14]:
# Passenger Volume by Origin Destination Train Stations

train_volume = call_api("https://datamall2.mytransport.sg/ltaodataservice/PV/ODTrain", api_key)
print(train_volume['Link'].tolist())

train_volume = pd.read_csv('origin_destination_train_202409.csv')
train_volume

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_train_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEL3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkcwRQIgZ%2Fnuit0AzSDnkig8i%2BhDxwki4%2FeXJm8NVizsjdGMkyECIQCPkNA%2BrJ5r%2FuoxuUgSt6fLeojzrLihB5mdnSGwI1aqfCrCBQgmEAQaDDM0MDY0NTM4MTMwNCIMZ4PJ6xzF83eMnwpCKp8FtSzg4tQvf6P%2F1QBLLZUiLVTyBNWVoz2ffwgAT%2BoflvGtrA1A%2FsGgCMZO%2FUhZQqxRK7opjrV%2Fd5AhwkoiHmfaobMX0%2B9b2CG1w6Qm9TKg8v45Ws7LA0E3na4WLryqLdVdy5DoTiMgAnpgUQ0LLdlEm60KJcguEZdMDyymuA3M4wjuEMxe78DcQy5niRyRFLWxO%2Fdp%2B%2F%2FxHgdI%2F0FUq0vqyFg9oQT8EonKEj0nOzA%2BXyFp8UtyYsO6Babf6q3Yq%2BLT8Zz0UARcstesZxvqKzDLddcbdZHvP6eQXDINk9EkADRr%2Bri41U07i77u6lkl5sitW6TEIcdy8wUXdmIshXL8pZ44ewvIdANyat2GxF7Xf6nxmM9HspBGk%2FFDOFesoFNpAg0i8QnylqZTOdqY%2BWfipYdg1O95%2BXeQYWrnfiSLnqBmJ2USA58aBFmSUepcgjasyN6wMAlFWm3SjF9lFxQFE1hYC294vwUFA3hOfb61LqFS2eEIV%2BZZ0Q6XGJDsuac2N7Fo0BK19L6kwKBp6dwtXUzn7BTs9szl3ltYxdajGzlDIQdVimdahjjHlLnOWuHJIhQriFsNPXIVO%2B5migxAwsNl4OfKZeMQb4Sbrv

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-09,WEEKDAY,6,TRAIN,NS18,NS13,484
1,2024-09,WEEKDAY,18,TRAIN,CC17/TE9,EW27,342
2,2024-09,WEEKDAY,15,TRAIN,EW28,NS3,351
3,2024-09,WEEKDAY,5,TRAIN,NS2,NS5,344
4,2024-09,WEEKDAY,18,TRAIN,TE15,TE22,225
...,...,...,...,...,...,...,...
867006,2024-09,WEEKENDS/HOLIDAY,19,TRAIN,PW4,DT22,1
867007,2024-09,WEEKDAY,16,TRAIN,NS21/DT11,BP13,1
867008,2024-09,WEEKENDS/HOLIDAY,18,TRAIN,EW20,CC12,1
867009,2024-09,WEEKDAY,11,TRAIN,PW5,NS8,1
